In [3697]:
import pandas as pd
import numpy as np
import networkx as nx
import plotly.graph_objects as go
import plotly.offline as pyo
import random


In [3698]:
# Load Dataframes from csv files
avg_counts_df = pd.read_csv('avg_counts_df.csv')
avg_rates_df = pd.read_csv('avg_rates_df.csv')
player_counts_df = avg_counts_df.copy()
player_counts_df

,Unnamed: 0,[0-0],[1-0],[0-1],[2-0],[1-1],[0-2],[3-0],[2-1],[1-2],[3-1],[2-2],[3-2],OUT,WALK,PLAY
0,[0-0],0.0,69861.0,91708.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20010.0
1,[1-0],0.0,0.0,0.0,23620.0,35117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10919.0
2,[0-1],0.0,0.0,0.0,0.0,36510.0,39148.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15716.0
3,[2-0],0.0,0.0,0.0,0.0,0.0,0.0,7236.0,12357.0,0.0,0.0,0.0,0.0,0.0,0.0,3907.0
4,[1-1],0.0,0.0,0.0,0.0,0.0,0.0,0.0,24406.0,32622.0,0.0,0.0,0.0,0.0,0.0,14320.0
5,[0-2],0.0,0.0,0.0,0.0,0.0,9238.0,0.0,0.0,21450.0,0.0,0.0,0.0,8652.0,0.0,8788.0
6,[3-0],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4473.0,0.0,0.0,0.0,2363.0,359.0
7,[2-1],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10756.0,17807.0,0.0,0.0,0.0,8089.0
8,[1-2],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15087.0,0.0,25876.0,0.0,13338.0,0.0,14493.0
9,[3-1],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7731.0,0.0,4160.0,3289.0


In [3699]:
player_counts = [player_counts_df.iloc[0]['[1-0]']+player_counts_df.iloc[0]['[0-1]']+player_counts_df.iloc[0]['PLAY'],
                 player_counts_df.iloc[1]['[2-0]']+player_counts_df.iloc[1]['[1-1]']+player_counts_df.iloc[1]['PLAY'],
                 player_counts_df.iloc[2]['[1-1]']+player_counts_df.iloc[2]['[0-2]']+player_counts_df.iloc[2]['PLAY'],
                 player_counts_df.iloc[3]['[3-0]']+player_counts_df.iloc[3]['[2-1]']+player_counts_df.iloc[3]['PLAY'],
                 player_counts_df.iloc[4]['[2-1]']+player_counts_df.iloc[4]['[1-2]']+player_counts_df.iloc[4]['PLAY'],
                 player_counts_df.iloc[5]['[0-2]']+player_counts_df.iloc[5]['[1-2]']+player_counts_df.iloc[5]['OUT']+player_counts_df.iloc[5]['PLAY'],
                 player_counts_df.iloc[6]['[3-1]']+player_counts_df.iloc[6]['WALK']+player_counts_df.iloc[6]['PLAY'],
                 player_counts_df.iloc[7]['[3-1]']+player_counts_df.iloc[7]['[2-2]']+player_counts_df.iloc[7]['PLAY'],
                 player_counts_df.iloc[8]['[1-2]']+player_counts_df.iloc[8]['[2-2]']+player_counts_df.iloc[8]['OUT']+player_counts_df.iloc[8]['PLAY'],
                 player_counts_df.iloc[9]['[3-2]']+player_counts_df.iloc[9]['WALK']+player_counts_df.iloc[9]['PLAY'],
                 player_counts_df.iloc[10]['[2-2]']+player_counts_df.iloc[10]['[3-2]']+player_counts_df.iloc[10]['OUT']+player_counts_df.iloc[10]['PLAY'],
                 player_counts_df.iloc[11]['[3-2]']+player_counts_df.iloc[11]['OUT']+player_counts_df.iloc[11]['WALK']+player_counts_df.iloc[11]['PLAY']]

avg_counts = [avg_counts_df.iloc[0]['[1-0]']+avg_counts_df.iloc[0]['[0-1]']+avg_counts_df.iloc[0]['PLAY'],
              avg_counts_df.iloc[1]['[2-0]']+avg_counts_df.iloc[1]['[1-1]']+avg_counts_df.iloc[1]['PLAY'],
              avg_counts_df.iloc[2]['[1-1]']+avg_counts_df.iloc[2]['[0-2]']+avg_counts_df.iloc[2]['PLAY'],
              avg_counts_df.iloc[3]['[3-0]']+avg_counts_df.iloc[3]['[2-1]']+avg_counts_df.iloc[3]['PLAY'],
              avg_counts_df.iloc[4]['[2-1]']+avg_counts_df.iloc[4]['[1-2]']+avg_counts_df.iloc[4]['PLAY'],
              avg_counts_df.iloc[5]['[0-2]']+avg_counts_df.iloc[5]['[1-2]']+avg_counts_df.iloc[5]['OUT']+avg_counts_df.iloc[5]['PLAY'],
              avg_counts_df.iloc[6]['[3-1]']+avg_counts_df.iloc[6]['WALK']+avg_counts_df.iloc[6]['PLAY'],
              avg_counts_df.iloc[7]['[3-1]']+avg_counts_df.iloc[7]['[2-2]']+avg_counts_df.iloc[7]['PLAY'],
              avg_counts_df.iloc[8]['[1-2]']+avg_counts_df.iloc[8]['[2-2]']+avg_counts_df.iloc[8]['OUT']+avg_counts_df.iloc[8]['PLAY'],
              avg_counts_df.iloc[9]['[3-2]']+avg_counts_df.iloc[9]['WALK']+avg_counts_df.iloc[9]['PLAY'],
              avg_counts_df.iloc[10]['[2-2]']+avg_counts_df.iloc[10]['[3-2]']+avg_counts_df.iloc[10]['OUT']+avg_counts_df.iloc[10]['PLAY'],
              avg_counts_df.iloc[11]['[3-2]']+avg_counts_df.iloc[11]['OUT']+avg_counts_df.iloc[11]['WALK']+avg_counts_df.iloc[11]['PLAY']]

player_counts = [int((i+random.uniform(-1000,1000))/100) for i in player_counts]
avg_counts = [int(i) for i in avg_counts]


In [3700]:
pct_avg_counts = [i/sum(avg_counts) for i in avg_counts]
pct_player_counts = [i/sum(player_counts) for i in player_counts]
count_diff = [(pct_player_counts[i]-pct_avg_counts[i])*100 for i in range(len(player_counts))]

In [3701]:
count_diff += [0, 0]
player_counts += [0, 0]

In [3702]:
player_counts = {
    '[0,0]': player_counts[0],
    '[1,0]': player_counts[1],
    '[0,1]': player_counts[2],
    '[2,0]': player_counts[3],
    '[1,1]': player_counts[4],
    '[0,2]': player_counts[5],
    '[3,0]': player_counts[6],
    '[2,1]': player_counts[7],
    '[1,2]': player_counts[8],
    '[3,1]': player_counts[9],
    '[2,2]': player_counts[10],
    '[3,2]': player_counts[11],
    'WALK': player_counts[12],
    'OUT': player_counts[13]
}
count_diff = {
    '[0,0]': count_diff[0],
    '[1,0]': count_diff[1],
    '[0,1]': count_diff[2],
    '[2,0]': count_diff[3],
    '[1,1]': count_diff[4],
    '[0,2]': count_diff[5],
    '[3,0]': count_diff[6],
    '[2,1]': count_diff[7],
    '[1,2]': count_diff[8],
    '[3,1]': count_diff[9],
    '[2,2]': count_diff[10],
    '[3,2]': count_diff[11],
    'WALK': count_diff[12],
    'OUT': count_diff[13]
}

In [3703]:
G = nx.DiGraph()
dec = 0
# Define edges between nodes that reflect state transition matrix
elist = [(1,2),(1,3),(2,4),(2,5),(3,5),(3,6),(4,7),(4,8),(5,8),(5,9),(6,6),(6,9),\
        (6,13),(7,10),(7,14),(8,10),(8,11),(9,9),(9,11),(9,13),(10,12),(10,14),\
        (11,11),(11,12),(11,13),(12,12),(12,13),(12,14)]

player_rates = [avg_rates_df.iloc[0]['[1-0]'],avg_rates_df.iloc[0]['[0-1]'],avg_rates_df.iloc[1]['[2-0]'],avg_rates_df.iloc[1]['[1-1]'],\
                 avg_rates_df.iloc[2]['[1-1]'],avg_rates_df.iloc[2]['[0-2]'],avg_rates_df.iloc[3]['[3-0]'],avg_rates_df.iloc[3]['[2-1]'],\
                 avg_rates_df.iloc[4]['[2-1]'],avg_rates_df.iloc[4]['[1-2]'],avg_rates_df.iloc[5]['[0-2]'],avg_rates_df.iloc[5]['[1-2]'],\
                 avg_rates_df.iloc[5]['OUT'],avg_rates_df.iloc[6]['[3-1]'],avg_rates_df.iloc[6]['WALK'],avg_rates_df.iloc[7]['[3-1]'],\
                 avg_rates_df.iloc[7]['[2-2]'],avg_rates_df.iloc[8]['[1-2]'],avg_rates_df.iloc[8]['[2-2]'],avg_rates_df.iloc[8]['OUT'],\
                 avg_rates_df.iloc[9]['[3-2]'],avg_rates_df.iloc[9]['WALK'],avg_rates_df.iloc[10]['[2-2]'],avg_rates_df.iloc[10]['[3-2]'],\
                 avg_rates_df.iloc[10]['OUT'],avg_rates_df.iloc[11]['[3-2]'],avg_rates_df.iloc[11]['OUT'],avg_rates_df.iloc[11]['WALK']]
player_rates = [element + random.uniform(-0.1, 0.1) for element in player_rates]

player_inplay_rates = [avg_rates_df.iloc[0]['PLAY'],avg_rates_df.iloc[1]['PLAY'],avg_rates_df.iloc[2]['PLAY'],avg_rates_df.iloc[3]['PLAY'],\
                       avg_rates_df.iloc[4]['PLAY'],avg_rates_df.iloc[5]['PLAY'],avg_rates_df.iloc[6]['PLAY'],avg_rates_df.iloc[7]['PLAY'],\
                       avg_rates_df.iloc[8]['PLAY'],avg_rates_df.iloc[9]['PLAY'],avg_rates_df.iloc[10]['PLAY'],avg_rates_df.iloc[11]['PLAY']]
player_inplay_rates = [element + random.uniform(-0.1, 0.1) for element in player_inplay_rates]

avg_rates = [avg_rates_df.iloc[0]['[1-0]'],avg_rates_df.iloc[0]['[0-1]'],avg_rates_df.iloc[1]['[2-0]'],avg_rates_df.iloc[1]['[1-1]'],\
              avg_rates_df.iloc[2]['[1-1]'],avg_rates_df.iloc[2]['[0-2]'],avg_rates_df.iloc[3]['[3-0]'],avg_rates_df.iloc[3]['[2-1]'],\
              avg_rates_df.iloc[4]['[2-1]'],avg_rates_df.iloc[4]['[1-2]'],avg_rates_df.iloc[5]['[0-2]'],avg_rates_df.iloc[5]['[1-2]'],\
              avg_rates_df.iloc[5]['OUT'],avg_rates_df.iloc[6]['[3-1]'],avg_rates_df.iloc[6]['WALK'],avg_rates_df.iloc[7]['[3-1]'],\
              avg_rates_df.iloc[7]['[2-2]'],avg_rates_df.iloc[8]['[1-2]'],avg_rates_df.iloc[8]['[2-2]'],avg_rates_df.iloc[8]['OUT'],\
              avg_rates_df.iloc[9]['[3-2]'],avg_rates_df.iloc[9]['WALK'],avg_rates_df.iloc[10]['[2-2]'],avg_rates_df.iloc[10]['[3-2]'],\
              avg_rates_df.iloc[10]['OUT'],avg_rates_df.iloc[11]['[3-2]'],avg_rates_df.iloc[11]['OUT'],avg_rates_df.iloc[11]['WALK']]

avg_inplay_rates = [avg_rates_df.iloc[0]['PLAY'],avg_rates_df.iloc[1]['PLAY'],avg_rates_df.iloc[2]['PLAY'],avg_rates_df.iloc[3]['PLAY'],\
                    avg_rates_df.iloc[4]['PLAY'],avg_rates_df.iloc[5]['PLAY'],avg_rates_df.iloc[6]['PLAY'],avg_rates_df.iloc[7]['PLAY'],\
                    avg_rates_df.iloc[8]['PLAY'],avg_rates_df.iloc[9]['PLAY'],avg_rates_df.iloc[10]['PLAY'],avg_rates_df.iloc[11]['PLAY']]

rate_diffs = [round((player_rates[i] - avg_rates[i])*100,dec) for i in range(len(player_rates))]

player_labels = [f"{int(round(element*100,dec))}%" for element in player_rates]
avg_labels = [f"{int(round(element*100,dec))}%" for element in avg_rates]

rate_labels = []
for rate_diff in rate_diffs:
    if rate_diff > 0:
        rate_labels.append(f"+{int(round(rate_diff,dec))}%")
    else:
        rate_labels.append(f"{int(round(rate_diff,dec))}%")

edges_with_labels = [(elist[i][0], elist[i][1], {'label': rate_labels[i]}) for i in range(len(elist))]


In [3704]:
good_idx = [0,1,0,1,0,1,0,1,0,1,0,0,1,1,0,0,1,0,0,1,1,0,0,0,1,0,1,0] # 1 = good, 0 = bad

In [3705]:
# Normalize rate_diffs to range between -1 and 1
normalized_diffs = 2 * (rate_diffs - np.min(rate_diffs)) / (np.max(rate_diffs) - np.min(rate_diffs)) - 1

color_intensities = []
for diff in normalized_diffs:
    color_intensities.append(int(255*diff))

In [3706]:
G.add_edges_from(edges_with_labels)
label_mapping = {1:'[0,0]',2:'[1,0]',3:'[0,1]',4:'[2,0]',5:'[1,1]',6:'[0,2]',\
                 7:'[3,0]',8:'[2,1]',9:'[1,2]',10:'[3,1]',11:'[2,2]',12:'[3,2]',\
                    13:'OUT',14:'WALK'}
G = nx.relabel_nodes(G, label_mapping)

In [3707]:
scale_factor = 1.5
pos = {
    '[0,0]': (150, 600*scale_factor),
    '[0,1]': (180, 500*scale_factor),
    '[1,0]': (120, 500*scale_factor),
    '[1,1]': (150, 400*scale_factor),
    '[0,2]': (210, 400*scale_factor),
    '[2,0]': (90, 400*scale_factor),
    '[1,2]': (180, 300*scale_factor),
    '[2,1]': (120, 300*scale_factor),
    '[2,2]': (150, 200*scale_factor),
    '[3,0]': (60, 300*scale_factor),
    '[3,1]': (90, 200*scale_factor),
    '[3,2]': (120, 100*scale_factor),
    'OUT': (210, 0),
    'WALK': (60, 0)
}

In [3708]:
# Create edge trace
def create_edge_trace(x0,y0,x1,y1,color,width):
    return  go.Scatter(
        x=[x0, x1, None],
        y=[y0, y1, None],
        mode='lines',
        line=dict(color=color, width=width),
        hoverinfo='none'
    )

edge_annotations = []

In [3709]:
edge_traces = []
loop_edge_colors = []
loop_text_colors = []
for i, edge in enumerate(G.edges()):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    if (good_idx[i] and color_intensities[i] > 0) or (not good_idx[i] and color_intensities[i] < 0):
        edge_color = f'rgba(255,{255-abs(color_intensities[i])},{255-abs(color_intensities[i])},1)'
        text_color = 'red'
    else:
        edge_color = f'rgba({255-abs(color_intensities[i])},{255-abs(color_intensities[i])},255,1)'
        text_color = 'blue'
    edge_trace = create_edge_trace(x0,y0,x1,y1,edge_color,width=3)
    edge_traces.append(edge_trace)

    if edge[0] == edge[1]:
        x_pos = x0+21
        y_pos = y0+5
        loop_edge_colors.append(edge_color)
        loop_text_colors.append(text_color)
    else:
        x_pos = (x0+x1)/2
        y_pos = (y0+y1)/2
    # Edge annotations (midpoint for label placement)
    edge_annotations.append(
        dict(
            x=x_pos,
            y=y_pos,
            xref='x',
            yref='y',
            text = '<b><i>'+str(G.edges[edge]['label'])+'</b></i>',
            showarrow=False,
            font=dict(color=text_color, size=14, family='Arial'),
            bgcolor='white',
            borderpad=0
        )
    )

In [3710]:
#static edge trace
edge_traces_single_color = []
for i, edge in enumerate(G.edges()):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_color = 'rgba(0, 0, 0, 0.2)'  # Change the color here
    edge_trace_single_color = create_edge_trace(x0, y0, x1, y1, edge_color, width=3)
    edge_traces_single_color.append(edge_trace_single_color)


In [3711]:
# Create node trace
node_trace = go.Scatter(
    x=[],
    y=[],
    mode='markers+text',
    hoverinfo='none',
    marker=dict(
        showscale=False,
        color=[],
        size=[],
        symbol='circle',
        line_width=0.5,
        line_color='black',
        opacity=1),
    text=[],
    textposition="middle center",
    textfont=dict(
        family="Arial",
        size=18,
        color="black"
    )
)

node_diameter = 85
# Add nodes to node trace
for i,node in enumerate(G.nodes()):
    x,y = pos[node]
    node_trace['x'] += tuple([x])
    node_trace['y'] += tuple([y])
    # Customize node size and color
    node_trace['marker']['color'] += tuple(['rgb(220,220,220,1)'])
    node_trace['marker']['size'] += tuple([node_diameter + 100*count_diff[node]])
    if node == 'OUT' or node == 'WALK':
        node_trace['text'] += tuple([str(node)])
    else:
        node_trace['text'] += tuple([str(node)[1]+'-'+str(node)[-2]])

In [3712]:
inplay_rate_diffs = [round((player_inplay_rates[i] - avg_inplay_rates[i])*100,dec) for i in range(len(player_inplay_rates))]
inplay_rate_labels = []

for inplay_rate_diff in inplay_rate_diffs:
    if inplay_rate_diff > 0:
        inplay_rate_labels.append(f"+{int(round(inplay_rate_diff,dec))}%")
    else:
        inplay_rate_labels.append(f"{int(round(inplay_rate_diff,dec))}%")

inplay_rate_labels = {
    '[0,0]': inplay_rate_labels[0],
    '[1,0]': inplay_rate_labels[1],
    '[0,1]': inplay_rate_labels[2],
    '[2,0]': inplay_rate_labels[3],
    '[1,1]': inplay_rate_labels[4],
    '[0,2]': inplay_rate_labels[5],
    '[3,0]': inplay_rate_labels[6],
    '[2,1]': inplay_rate_labels[7],
    '[1,2]': inplay_rate_labels[8],
    '[3,1]': inplay_rate_labels[9],
    '[2,2]': inplay_rate_labels[10],
    '[3,2]': inplay_rate_labels[11],
    'WALK': "0",
    'OUT': "0"
}

In [3713]:
# Create text trace
text_trace = go.Scatter(
    x=[],
    y=[],
    mode='text',
    hoverinfo='none',
    text=[],
    textposition="middle center",
    textfont=dict(
        family="Arial",
        size=10,
        color="black"
    )
)

for node in G.nodes():
    x,y = pos[node]
    text_trace['x'] += tuple([x])
    text_trace['y'] += tuple([y])
    text_trace['text'] += tuple(['<br></br>'+'IN PLAY:'+'<b>'+str(inplay_rate_labels[node])+'</b>'])

In [3714]:
traces =  edge_traces + edge_traces_single_color + [node_trace] + [text_trace]

In [3715]:
fig = go.Figure(data=traces,
                layout=go.Layout(
                    width=900,
                    height=900,
                    paper_bgcolor='rgba(0,0,0,0)',
                    plot_bgcolor='rgba(0,0,0,0)',
                    titlefont_size=16,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=0, l=0, r=0, t=0, pad=0),  # Update the margin values here
                    annotations=edge_annotations,
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False, scaleanchor="y", scaleratio=5, fixedrange=True),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False, scaleanchor="x", scaleratio=1, fixedrange=True)))

In [3716]:
# Add loops to graph with arrow caps
loops = []
arrows = []
two_strike_index = [5,8,12,13]
for i in range(4):
    node_size = node_trace['marker']['size'][two_strike_index[i]]/10
    loop = dict(
        start_pt = [pos[f'[{i},2]'][0]+node_size+1, pos[f'[{i},2]'][1]+node_size+10],
        end_pt = [pos[f'[{i},2]'][0]+node_size+2, pos[f'[{i},2]'][1]+node_size-20],
        control_pt1 = [pos[f'[{i},2]'][0]+node_size+15, pos[f'[{i},2]'][1]+node_size+15],
        control_pt2 = [pos[f'[{i},2]'][0]+node_size+5, pos[f'[{i},2]'][1]+node_size-30]
    )
    loops.append(loop)
    
    arrow = dict(
        type="path",
        path=f"M {loop['end_pt'][0]-0.5},{loop['end_pt'][1]-1.5} L {loop['end_pt'][0]+1.5},{loop['end_pt'][1]+8.5} L {loop['end_pt'][0]+2},{loop['end_pt'][1]-5} Z",
        fillcolor=loop_edge_colors[i],
        line=dict(color=loop_edge_colors[i]),
        line_width=0
    )
    arrows.append(arrow)
    
    arrow = dict(
    type="path",
    path=f"M {loop['end_pt'][0]-0.5},{loop['end_pt'][1]-1.5} L {loop['end_pt'][0]+1.5},{loop['end_pt'][1]+8.5} L {loop['end_pt'][0]+2},{loop['end_pt'][1]-5} Z",
    fillcolor="rgba(0,0,0,0.2)",
    line=dict(color="rgba(0,0,0,0.2)"),
    line_width=0
    )
    arrows.append(arrow)
    
shapes = []
for i,loop in enumerate(loops):
    shapes.append(
        dict(
            type="path",
            path=f"M {loop['start_pt'][0]},{loop['start_pt'][1]} C {loop['control_pt1'][0]},{loop['control_pt1'][1]} {loop['control_pt2'][0]},{loop['control_pt2'][1]} {loop['end_pt'][0]},{loop['end_pt'][1]}",
            line_color=loop_edge_colors[i],
        )
    )
    shapes.append(
        dict(
            type="path",
            path=f"M {loop['start_pt'][0]},{loop['start_pt'][1]} C {loop['control_pt1'][0]},{loop['control_pt1'][1]} {loop['control_pt2'][0]},{loop['control_pt2'][1]} {loop['end_pt'][0]},{loop['end_pt'][1]}",
            line_color="rgba(0,0,0,0.2)"
        )
    )

shapes.extend(arrows)
fig.update_layout(shapes=shapes)

In [3717]:
from dash import Dash, html, dcc, Input, Output, callback

In [3718]:

app = Dash(__name__)
app.layout = html.Div(children=[
    html.Div('Pitching with Markov', style={'font-family': 'Arial', 'font-size': '24px', 'font-style': 'italic'}),
    html.Div([
        dcc.Dropdown(
        id='player-dropdown',
        options=[
            {'label': 'Clayton Kershaw', 'value': 'kershaw'},
            {'label': 'Lucas Giolito', 'value': 'giolito'},
            {'label': 'Nathan Eovaldi', 'value': 'eovaldi'},
            {'label': 'League Average', 'value': 'average'}
        ],
        value='average'
        )
    ], style={'width': '50%', 'display': 'inline-block','font-family': 'Arial', 'font-size': '16px'}),
    dcc.Graph(
        id='example-graph',
        figure=fig)
])


In [3719]:
app.run(debug=True)
